# Graph-Learning-Based Recommender System on MovieLens

### Group 9

- AGARWAL, Sahil
- WEI, Yuanjing
- ZHANG, Yujun yzhanglo@connect.ust.hk

Group project of COMP4222@HKUST in 2022 Fall.

# 1 Environment Configuration

In [ ]:
# change the path in the following
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/4222Group9'
except:
    %cd '/data/yzhanglo/4222project'

import comp4222
import recommenders
%pwd
%ls

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tensorboardX import SummaryWriter

import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

# easier to print by putting variable as a single line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# make matplotlib figures appear inline in the notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Testing the module functionality
from comp4222 import b
comp4222.b.ok()

# 2 MovieLens


We're using ml-latest-small from MovieLens. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018. The readme.md is avaliable [here](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

## Data Loading for ml-latest-small

In [ ]:
# Download MovieLens data.
dataset_name = "ml-latest-small"
from urllib.request import urlretrieve
import zipfile
urlretrieve(f"https://files.grouplens.org/datasets/movielens/{dataset_name}.zip", "movielens.zip")
zipfile.ZipFile("movielens.zip", "r").extractall()

In [ ]:
movies = pd.read_csv(f"{dataset_name}/movies.csv")
genre_cols = [
    "(no genres listed)", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies

In [ ]:
tags = pd.read_csv(f"{dataset_name}/tags.csv")
tags

In [ ]:
ratings = pd.read_csv(f"{dataset_name}/ratings.csv")
ratings

# 5 Backup Models Definition

In [ ]:
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.utils.constants import (
    COL_DICT,
    DEFAULT_K,
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
    DEFAULT_TIMESTAMP_COL,
    SEED,
)

# Helpers
import os
from tempfile import TemporaryDirectory
tmp_dir = TemporaryDirectory()
TRAIN_FILE = os.path.join(tmp_dir.name, "df_train.csv")
TEST_FILE = os.path.join(tmp_dir.name, "df_test.csv")

In [ ]:
from recommenders.utils.spark_utils import start_or_get_spark
spark = start_or_get_spark("PySpark", memory="32g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split

# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

### ALS

In [ ]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType
from pyspark.ml.recommendation import ALS


def prepare_training_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache()

def prepare_metrics_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache(), spark.createDataFrame(test, schema).cache()

def predict_als(model, test):
    with Timer() as t:
        preds = model.transform(test)
    return preds, t

def train_als(params, data):
    symbol = ALS(**params)
    with Timer() as t:
        model = symbol.fit(data)
    return model, t

def recommend_k_als(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        # Get the cross join of all user-item pairs and score them.
        users = train.select(DEFAULT_USER_COL).distinct()
        items = train.select(DEFAULT_ITEM_COL).distinct()
        user_item = users.crossJoin(items)
        dfs_pred = model.transform(user_item)

        # Remove seen items
        dfs_pred_exclude_train = dfs_pred.alias("pred").join(
            train.alias("train"),
            (dfs_pred[DEFAULT_USER_COL] == train[DEFAULT_USER_COL])
            & (dfs_pred[DEFAULT_ITEM_COL] == train[DEFAULT_ITEM_COL]),
            how="outer",
        )
        topk_scores = dfs_pred_exclude_train.filter(
            dfs_pred_exclude_train["train." + DEFAULT_RATING_COL].isNull()
        ).select(
            "pred." + DEFAULT_USER_COL,
            "pred." + DEFAULT_ITEM_COL,
            "pred." + DEFAULT_PREDICTION_COL,
        )
    return topk_scores, t


als_params = {
    "rank": 10,
    "maxIter": 20,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": DEFAULT_USER_COL,
    "itemCol": DEFAULT_ITEM_COL,
    "ratingCol": DEFAULT_RATING_COL,
}

### NCF

In [ ]:
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset

def prepare_training_ncf(df_train, df_test):
    #df_train.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    #df_test.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    train = df_train.sort_values(["userID"], axis=0, ascending=[True])
    test = df_test.sort_values(["userID"], axis=0, ascending=[True])
    test = test[df_test["userID"].isin(train["userID"].unique())]
    test = test[test["itemID"].isin(train["itemID"].unique())]
    train.to_csv(TRAIN_FILE, index=False)
    test.to_csv(TEST_FILE, index=False)
    return NCFDataset(
        train_file=TRAIN_FILE,
        col_user=DEFAULT_USER_COL,
        col_item=DEFAULT_ITEM_COL,
        col_rating=DEFAULT_RATING_COL,
        seed=SEED,
    )


def train_ncf(params, data):
    model = NCF(n_users=data.n_users, n_items=data.n_items, **params)
    with Timer() as t:
        model.fit(data)
    return model, t


def recommend_k_ncf(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        users, items, preds = [], [], []
        item = list(train[DEFAULT_ITEM_COL].unique())
        for user in train[DEFAULT_USER_COL].unique():
            user = [user] * len(item)
            users.extend(user)
            items.extend(item)
            preds.extend(list(model.predict(user, item, is_list=True)))
        topk_scores = pd.DataFrame(
            data={
                DEFAULT_USER_COL: users,
                DEFAULT_ITEM_COL: items,
                DEFAULT_PREDICTION_COL: preds,
            }
        )
        merged = pd.merge(
            train, topk_scores, on=[DEFAULT_USER_COL, DEFAULT_ITEM_COL], how="outer"
        )
        topk_scores = merged[merged[DEFAULT_RATING_COL].isnull()].drop(
            DEFAULT_RATING_COL, axis=1
        )
    # Remove temp files
    return topk_scores, t

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 20,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

### KGAT

In [ ]:
#%pip install easydict
import os
import sys
import random
from time import time

import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

from KGAT_folder.KGAT import KGAT
from KGAT_folder.log_helper import *
from KGAT_folder.parser_kgat import *
from KGAT_folder.metrics import *
from KGAT_folder.model_helper import *
from KGAT_folder.loader_kgat import DataLoaderKGAT


def train_kgat():
    args = parse_kgat_args()
    #train(args)
    time0 = time()
    model, data, Ks, device = train(args)
    time1 = time()
    t = time1-time0
    return model, data, Ks, device, t

def evaluate_kgat(model, dataloader, Ks, device):
    test_batch_size = dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict

    model.eval()

    user_ids = list(test_user_dict.keys())
    user_ids_batches = [user_ids[i: i + test_batch_size] for i in range(0, len(user_ids), test_batch_size)]
    user_ids_batches = [torch.LongTensor(d) for d in user_ids_batches]

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)

    cf_scores = []
    metric_names = ['precision', 'recall', 'ndcg']
    metrics_dict = {k: {m: [] for m in metric_names} for k in Ks}

    with tqdm(total=len(user_ids_batches), desc='Evaluating Iteration') as pbar:
        for batch_user_ids in user_ids_batches:
            batch_user_ids = batch_user_ids.to(device)

            with torch.no_grad():
                batch_scores = model(batch_user_ids, item_ids, mode='predict')       # (n_batch_users, n_items)

            batch_scores = batch_scores.cpu()
            batch_metrics = calc_metrics_at_k(batch_scores, train_user_dict, test_user_dict, batch_user_ids.cpu().numpy(), item_ids.cpu().numpy(), Ks)

            cf_scores.append(batch_scores.numpy())
            for k in Ks:
                for m in metric_names:
                    metrics_dict[k][m].append(batch_metrics[k][m])
            pbar.update(1)

    cf_scores = np.concatenate(cf_scores, axis=0)
    for k in Ks:
        for m in metric_names:
            metrics_dict[k][m] = np.concatenate(metrics_dict[k][m]).mean()
    return cf_scores, metrics_dict


def train(args):
    # seed
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    #log_save_id = create_log_id(args.save_dir)
    #logging_config(folder=args.save_dir, name='log{:d}'.format(log_save_id), no_console=False)
    #logging.info(args)

    # GPU / CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load data
    data = DataLoaderKGAT(args, logging)
    if args.use_pretrain == 1:
        user_pre_embed = torch.tensor(data.user_pre_embed)
        item_pre_embed = torch.tensor(data.item_pre_embed)
    else:
        user_pre_embed, item_pre_embed = None, None

    # construct model & optimizer
    model = KGAT(args, data.n_users, data.n_entities, data.n_relations, data.A_in, user_pre_embed, item_pre_embed)
    #if args.use_pretrain == 2:
    #    model = load_model(model, args.pretrain_model_path)

    model.to(device)
    #logging.info(model)

    cf_optimizer = optim.Adam(model.parameters(), lr=args.lr)
    kg_optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # initialize metrics
    best_epoch = -1
    best_recall = 0

    Ks = eval(args.Ks)
    k_min = min(Ks)
    k_max = max(Ks)

    epoch_list = []
    metrics_list = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in Ks}

    # train model
    for epoch in range(1, args.n_epoch + 1):
        time0 = time()
        model.train()

        # train cf
        time1 = time()
        cf_total_loss = 0
        n_cf_batch = data.n_cf_train // data.cf_batch_size + 1

        for iter in range(1, n_cf_batch + 1):
            time2 = time()
            cf_batch_user, cf_batch_pos_item, cf_batch_neg_item = data.generate_cf_batch(data.train_user_dict, data.cf_batch_size)
            cf_batch_user = cf_batch_user.to(device)
            cf_batch_pos_item = cf_batch_pos_item.to(device)
            cf_batch_neg_item = cf_batch_neg_item.to(device)

            cf_batch_loss = model(cf_batch_user, cf_batch_pos_item, cf_batch_neg_item, mode='train_cf')

            if np.isnan(cf_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (CF Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_cf_batch))
                sys.exit()

            cf_batch_loss.backward()
            cf_optimizer.step()
            cf_optimizer.zero_grad()
            cf_total_loss += cf_batch_loss.item()

            if (iter % args.cf_print_every) == 0:
                logging.info('CF Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_cf_batch, time() - time2, cf_batch_loss.item(), cf_total_loss / iter))
        logging.info('CF Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_cf_batch, time() - time1, cf_total_loss / n_cf_batch))

        # train kg
        time3 = time()
        kg_total_loss = 0
        n_kg_batch = data.n_kg_train // data.kg_batch_size + 1

        for iter in range(1, n_kg_batch + 1):
            time4 = time()
            kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail = data.generate_kg_batch(data.train_kg_dict, data.kg_batch_size, data.n_users_entities)
            kg_batch_head = kg_batch_head.to(device)
            kg_batch_relation = kg_batch_relation.to(device)
            kg_batch_pos_tail = kg_batch_pos_tail.to(device)
            kg_batch_neg_tail = kg_batch_neg_tail.to(device)

            kg_batch_loss = model(kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail, mode='train_kg')

            if np.isnan(kg_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (KG Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_kg_batch))
                sys.exit()

            kg_batch_loss.backward()
            kg_optimizer.step()
            kg_optimizer.zero_grad()
            kg_total_loss += kg_batch_loss.item()

            if (iter % args.kg_print_every) == 0:
                logging.info('KG Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_kg_batch, time() - time4, kg_batch_loss.item(), kg_total_loss / iter))
        logging.info('KG Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_kg_batch, time() - time3, kg_total_loss / n_kg_batch))

        # update attention
        time5 = time()
        h_list = data.h_list.to(device)
        t_list = data.t_list.to(device)
        r_list = data.r_list.to(device)
        relations = list(data.laplacian_dict.keys())
        model(h_list, t_list, r_list, relations, mode='update_att')
        logging.info('Update Attention: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time5))

        logging.info('CF + KG Training: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time0))
    return model, data, Ks, device


In [ ]:
print(f"\nComputing {'KGAT'} algorithm on Movielens {'100k'}")
model, data, Ks, device, time_train = train_kgat()
print('time_train: ', time_train)
_, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
print(metrics_dict_kgat)

### LightGCN

In [ ]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
    
def prepare_training_lightgcn(train, test):
    return ImplicitCF(train=train, test=test)

def train_lightgcn(params, data):
    hparams = prepare_hparams(**params)
    model = LightGCN(hparams, data)
    with Timer() as t:
        model.fit()
    return model, t

def recommend_k_lightgcn(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        topk_scores = model.recommend_k_items(
            test, top_k=top_k, remove_seen=remove_seen
        )
    return topk_scores, t

lightgcn_param = {
    #"yaml_file": os.path.join("drive", "MyDrive", "4222Group9", "lightgcn.yaml"),
    "yaml_file": "lightgcn.yaml",
    "n_layers": 3,
    "batch_size": 1024,
    "epochs": 10,
    "learning_rate": 0.005,
    "eval_epoch": 5,
    "top_k": DEFAULT_K,
    "stacking_func": 1, #0: original, 1: exponential decay, 1.5: exponential increase, 2: trainable with random initialization, 3: trainable with unified initialization
}

# Prepare for training

In [ ]:


from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
)
def ranking_metrics_python(test, predictions, k=DEFAULT_K):
    return {
        "MAP": map_at_k(test, predictions, k=k, **COL_DICT),
        "nDCG@k": ndcg_at_k(test, predictions, k=k, **COL_DICT),
        "Precision@k": precision_at_k(test, predictions, k=k, **COL_DICT),
        "Recall@k": recall_at_k(test, predictions, k=k, **COL_DICT),
    }

#params = {
#    "als": als_params,
#    "ncf": ncf_params,
#    "lightgcn": lightgcn_param,
#}
prepare_training_data = {
    "als": prepare_training_als,
    "ncf": prepare_training_ncf,
    "lightgcn": prepare_training_lightgcn,
}


from recommenders.evaluation.spark_evaluation import (
    SparkRatingEvaluation,
    SparkRankingEvaluation,
)
def rating_metrics_pyspark(test, predictions):
    rating_eval = SparkRatingEvaluation(test, predictions, **COL_DICT)
    return {
        "RMSE": rating_eval.rmse(),
        "MAE": rating_eval.mae(),
        "R2": rating_eval.exp_var(),
        "Explained Variance": rating_eval.rsquared(),
    }
def ranking_metrics_pyspark(test, predictions, k=DEFAULT_K):
    rank_eval = SparkRankingEvaluation(
        test, predictions, k=k, relevancy_method="top_k", **COL_DICT
    )
    return {
        "MAP": rank_eval.map_at_k(),
        "nDCG@k": rank_eval.ndcg_at_k(),
        "Precision@k": rank_eval.precision_at_k(),
        "Recall@k": rank_eval.recall_at_k(),
    }

trainer = {
    "als": lambda params, data: train_als(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}
ranking_predictor = {
    "als": lambda model, test, train: recommend_k_als(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}
ranking_evaluator = {
    "als": lambda test, predictions, k: ranking_metrics_pyspark(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}
metrics = {
    #"als": ["rating", "ranking"],
    "als" : ["ranking"],
    "ncf": ["ranking"],
    "lightgcn": ["ranking"]
}

In [ ]:
from recommenders.evaluation.spark_evaluation import (
    SparkRatingEvaluation,
    SparkRankingEvaluation,
)
def rating_metrics_pyspark(test, predictions):
    rating_eval = SparkRatingEvaluation(test, predictions, **COL_DICT)
    return {
        "RMSE": rating_eval.rmse(),
        "MAE": rating_eval.mae(),
        "R2": rating_eval.exp_var(),
        "Explained Variance": rating_eval.rsquared(),
    }
def ranking_metrics_pyspark(test, predictions, k=DEFAULT_K):
    rank_eval = SparkRankingEvaluation(
        test, predictions, k=k, relevancy_method="top_k", **COL_DICT
    )
    return {
        "MAP": rank_eval.map_at_k(),
        "nDCG@k": rank_eval.ndcg_at_k(),
        "Precision@k": rank_eval.precision_at_k(),
        "Recall@k": rank_eval.recall_at_k(),
    }
trainer = {
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}
ranking_predictor = {
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}
ranking_evaluator = {
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}
metrics = {
    "lightgcn": ["ranking"]
}

## Sanity Check by Overfitting on Small Data

In [ ]:
#!pip install pytorch-lightning
#from pytorch_lightning import Trainer, seed_everything
#from LightGCN.code.main_lgcn import sanity_check

#seed_everything(42, workers=True)

#model = sanity_check()
#trainer = Trainer(max_epochs=10000, overfit_batches=0.01)
#trainer.fit(model)

# 6 Hyperparameter Tunning

In [ ]:
def generate_summary(data, algo, k, train_time, time_rating, rating_metrics, time_ranking, ranking_metrics):
    summary = {"Data": data, "K": k, "Train time (s)": train_time, "Recommending time (s)": time_ranking}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(ranking_metrics)
    return summary

In [42]:
data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

lightgcn_param_dict = {
    #"yaml_file": os.path.join("drive", "MyDrive", "4222Group9", "lightgcn.yaml"),
    "yaml_file": "lightgcn.yaml",
    "embed_size" : [32, 64, 128], # the embedding dimension of users and items
    # "n_layers": 3, # number of layers of the model

    # "batch_size": 1024,
    "decay" : 0.0001, # l2 regularization for embedding parameters
    "epochs": 200,
    "learning_rate": [0.0005, 0.001, 0.002],
    "eval_epoch": 50,
    # "top_k": DEFAULT_K,
    "stacking_func": [0,1,1.5] #0: original, 1: exponential decay, 1.5: exponential increase, 2: trainable with random initialization, 3: trainable with unified initialization

    # "save_model" : False,
    # "save_epoch" : 10
}

In [43]:
from recommenders.tuning.parameter_sweep import generate_param_grid
lightgcn_params = generate_param_grid(lightgcn_param_dict)

### Training

In [44]:
import traceback
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split

In [45]:
%%time
data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)",  "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:

    # Load the dataset
    #df = movielens.load_pandas_df(
    #    size=data_size,
    #    header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    #)
    if data_size=='100k':
        df = pd.read_csv('ml-100k/u.data', sep='\t', header=None)

    df.columns = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    df[DEFAULT_RATING_COL] = df[DEFAULT_RATING_COL].astype(float)
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL,
                                                )

    #df_train = "../LightGCN/data/movielens/train.txt"
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
        if algo == 'kgat':
            model, data, Ks, device, time_train = train_kgat()
            _, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
            print(metrics_dict_kgat)
            # Record results
            #summary = generate_summary('100k', algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            #df_results.loc[df_results.shape[0] + 1] = summary
            
        else:
            # Data prep for training set
            obj = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            for model_params in lightgcn_params:
                print("training using the params:", model_params)

                # Train the model
                #try:
                model, time_train = trainer[algo](model_params, obj)
                #except:
                    #traceback.print_exc()
                
                print(f"Training time: {time_train}s")
                        
                # Predict and evaluate
                #train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
                train, test = df_train, df_test
                
                if "rating" in metrics[algo]:   
                    pass
                else:
                    ratings = None
                    time_rating = np.nan
                
                if "ranking" in metrics[algo]:
                    # Predict for ranking
                    top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
                    print(f"Ranking prediction time: {time_ranking}s")
                    
                    # Evaluate for rating
                    rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
                else:
                    rankings = None
                    time_ranking = np.nan
                    
                # Record results
                summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
                df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation finished")

Size of Movielens 100k: (100000, 4)

Computing lightgcn algorithm on Movielens 100k
training using the params: {'embed_size': 32, 'learning_rate': 0.0005, 'stacking_func': 0, 'yaml_file': 'lightgcn.yaml', 'decay': 0.0001, 'epochs': 200, 'eval_epoch': 50}
Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2022-11-21 23:28:47.820270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 23:28:47.820913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-11-21 23:28:47.821153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 23:28:47.821704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0c:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMem

Epoch 1 (train)3.2s: train loss = 0.69160 = (mf)0.69159 + (embed)0.00001
Epoch 2 (train)0.7s: train loss = 0.66722 = (mf)0.66720 + (embed)0.00002
Epoch 3 (train)0.8s: train loss = 0.59980 = (mf)0.59974 + (embed)0.00006
Epoch 4 (train)0.7s: train loss = 0.51820 = (mf)0.51809 + (embed)0.00011
Epoch 5 (train)0.7s: train loss = 0.45033 = (mf)0.45016 + (embed)0.00017
Epoch 6 (train)0.7s: train loss = 0.40087 = (mf)0.40064 + (embed)0.00023
Epoch 7 (train)0.7s: train loss = 0.36988 = (mf)0.36960 + (embed)0.00028
Epoch 8 (train)0.7s: train loss = 0.35035 = (mf)0.35002 + (embed)0.00033
Epoch 9 (train)0.7s: train loss = 0.33893 = (mf)0.33857 + (embed)0.00037
Epoch 10 (train)0.7s: train loss = 0.32843 = (mf)0.32804 + (embed)0.00040
Epoch 11 (train)0.7s: train loss = 0.32753 = (mf)0.32710 + (embed)0.00043
Epoch 12 (train)0.8s: train loss = 0.32154 = (mf)0.32109 + (embed)0.00045
Epoch 13 (train)0.7s: train loss = 0.32045 = (mf)0.31998 + (embed)0.00047
Epoch 14 (train)0.8s: train loss = 0.31629 = (m

/data/yzhanglo/4222project/recommenders/models/deeprec/models/graphrec/lightgcn.py:170: SymPyDeprecationWarning: 

Eq(expr) with a single argument with the right-hand side
defaulting to 0 is deprecated. Use Eq(expr, 0) instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-eq-expr
for details.

This has been deprecated since SymPy version 1.5. It
will be removed in a future version of SymPy.

  eq1 = Eq(1/x+1/x**2+1/x**3+1/x**4-1)
2022-11-21 23:31:26.176509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 23:31:26.177114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-11-21 23:3

Epoch 1 (train)3.4s: train loss = 0.69293 = (mf)0.69292 + (embed)0.00001
Epoch 2 (train)0.8s: train loss = 0.69221 = (mf)0.69220 + (embed)0.00001
Epoch 3 (train)0.9s: train loss = 0.68871 = (mf)0.68870 + (embed)0.00001
Epoch 4 (train)0.7s: train loss = 0.67390 = (mf)0.67388 + (embed)0.00002
Epoch 5 (train)0.7s: train loss = 0.63538 = (mf)0.63534 + (embed)0.00004
Epoch 6 (train)0.7s: train loss = 0.57328 = (mf)0.57321 + (embed)0.00008
Epoch 7 (train)0.7s: train loss = 0.50263 = (mf)0.50250 + (embed)0.00013
Epoch 8 (train)0.7s: train loss = 0.44217 = (mf)0.44199 + (embed)0.00018
Epoch 9 (train)0.9s: train loss = 0.39348 = (mf)0.39325 + (embed)0.00023
Epoch 10 (train)0.8s: train loss = 0.36148 = (mf)0.36119 + (embed)0.00028
Epoch 11 (train)0.8s: train loss = 0.33603 = (mf)0.33570 + (embed)0.00033
Epoch 12 (train)0.9s: train loss = 0.31737 = (mf)0.31700 + (embed)0.00037
Epoch 13 (train)0.8s: train loss = 0.30743 = (mf)0.30703 + (embed)0.00040
Epoch 14 (train)0.7s: train loss = 0.29886 = (m

...

Epoch 1 (train)4.6s: train loss = 0.69317 = (mf)0.69315 + (embed)0.00002
Epoch 2 (train)0.9s: train loss = 0.69317 = (mf)0.69315 + (embed)0.00002
Epoch 3 (train)0.8s: train loss = 0.69316 = (mf)0.69315 + (embed)0.00002
Epoch 4 (train)0.9s: train loss = 0.69316 = (mf)0.69315 + (embed)0.00001
Epoch 5 (train)0.8s: train loss = 0.69316 = (mf)0.69315 + (embed)0.00001
Epoch 6 (train)0.9s: train loss = 0.69315 = (mf)0.69314 + (embed)0.00001
Epoch 7 (train)0.9s: train loss = 0.69315 = (mf)0.69314 + (embed)0.00001
Epoch 8 (train)0.9s: train loss = 0.69314 = (mf)0.69313 + (embed)0.00001
Epoch 9 (train)0.8s: train loss = 0.69313 = (mf)0.69311 + (embed)0.00002
Epoch 10 (train)0.9s: train loss = 0.69309 = (mf)0.69306 + (embed)0.00003
Epoch 11 (train)1.0s: train loss = 0.69302 = (mf)0.69295 + (embed)0.00007
Epoch 12 (train)0.9s: train loss = 0.69288 = (mf)0.69275 + (embed)0.00013
Epoch 13 (train)0.9s: train loss = 0.69264 = (mf)0.69240 + (embed)0.00024
Epoch 14 (train)0.9s: train loss = 0.69228 = (m

2022-11-22 00:27:51.290479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 00:27:51.291096: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-11-22 00:27:51.291310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 00:27:51.291705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0c:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMem

Epoch 1 (train)4.9s: train loss = 0.65891 = (mf)0.65886 + (embed)0.00004
Epoch 2 (train)0.9s: train loss = 0.43099 = (mf)0.43076 + (embed)0.00022
Epoch 3 (train)0.9s: train loss = 0.32721 = (mf)0.32678 + (embed)0.00043
Epoch 4 (train)0.9s: train loss = 0.30727 = (mf)0.30674 + (embed)0.00053
Epoch 5 (train)0.9s: train loss = 0.29051 = (mf)0.28992 + (embed)0.00059
Epoch 6 (train)0.9s: train loss = 0.27957 = (mf)0.27893 + (embed)0.00064
Epoch 7 (train)0.8s: train loss = 0.26882 = (mf)0.26812 + (embed)0.00069
Epoch 8 (train)0.9s: train loss = 0.25722 = (mf)0.25648 + (embed)0.00074
Epoch 9 (train)0.9s: train loss = 0.25330 = (mf)0.25250 + (embed)0.00079
Epoch 10 (train)0.9s: train loss = 0.24436 = (mf)0.24351 + (embed)0.00084
Epoch 11 (train)0.9s: train loss = 0.23804 = (mf)0.23715 + (embed)0.00089
Epoch 12 (train)0.9s: train loss = 0.23750 = (mf)0.23657 + (embed)0.00093
Epoch 13 (train)0.9s: train loss = 0.23490 = (mf)0.23392 + (embed)0.00097
Epoch 14 (train)0.9s: train loss = 0.23034 = (m

/data/yzhanglo/4222project/recommenders/models/deeprec/models/graphrec/lightgcn.py:170: SymPyDeprecationWarning: 

Eq(expr) with a single argument with the right-hand side
defaulting to 0 is deprecated. Use Eq(expr, 0) instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-eq-expr
for details.

This has been deprecated since SymPy version 1.5. It
will be removed in a future version of SymPy.

  eq1 = Eq(1/x+1/x**2+1/x**3+1/x**4-1)
2022-11-22 00:31:04.997562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 00:31:04.998249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-11-22 00:3

Epoch 1 (train)5.2s: train loss = 0.69072 = (mf)0.69069 + (embed)0.00003
Epoch 2 (train)0.9s: train loss = 0.64883 = (mf)0.64877 + (embed)0.00006
Epoch 3 (train)0.7s: train loss = 0.46747 = (mf)0.46728 + (embed)0.00019
Epoch 4 (train)0.9s: train loss = 0.32766 = (mf)0.32727 + (embed)0.00039
Epoch 5 (train)0.8s: train loss = 0.27989 = (mf)0.27937 + (embed)0.00053
Epoch 6 (train)0.9s: train loss = 0.25935 = (mf)0.25873 + (embed)0.00062
Epoch 7 (train)0.8s: train loss = 0.24327 = (mf)0.24259 + (embed)0.00069
Epoch 8 (train)0.9s: train loss = 0.23309 = (mf)0.23234 + (embed)0.00075
Epoch 9 (train)0.9s: train loss = 0.22368 = (mf)0.22288 + (embed)0.00080
Epoch 10 (train)0.8s: train loss = 0.21890 = (mf)0.21806 + (embed)0.00084
Epoch 11 (train)0.7s: train loss = 0.21135 = (mf)0.21047 + (embed)0.00088
Epoch 12 (train)0.8s: train loss = 0.20894 = (mf)0.20802 + (embed)0.00092
Epoch 13 (train)0.8s: train loss = 0.20065 = (mf)0.19970 + (embed)0.00096
Epoch 14 (train)0.7s: train loss = 0.19916 = (m

## Training Result Plot

Check the result table, and then filter on that table to see suitable params setting

In [ ]:
df_result = df_results
df_result['param'] = lightgcn_params
with pd.option_context('display.max_rows', None):
    with pd.option_context('display.max_colwidth', None):
        df_result.to_csv('params.csv')
        #from IPython.display import HTML
        #HTML(df_result.to_html())
        df_result

In [ ]:
lightgcn_params # see the corresponding param

# 7 Comparisons on Movielens and Movie

In [ ]:
data_sizes = ["100k","1m"] # Movielens data size: 100k, 1m, 10m, or 20m
#algorithms = [ "lightgcn"]
algorithms = ["als", "ncf", "lightgcn", "kgat"]

## Print the result summary

In [ ]:
df_results

# 8 Credit and Reference

1. https://github.com/microsoft/recommenders